<a href="https://colab.research.google.com/github/lilianiazzai/random/blob/main/IA_KevinBacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset e Criação do Grafo



In [16]:
import pandas as pd
import networkx as nx
import ast
import matplotlib.pyplot as plt

data = pd.read_csv("data.csv")

#Criação do grafo de kevin bacon
G = nx.Graph()

In [17]:
data.head()

,Unnamed: 0,Movie Name,Year of Release,Run Time in minutes,Movie Rating,Votes,MetaScore,Gross,Genre,Certification,Director,Stars,Description
0,0,The Shawshank Redemption,1994,142,9.3,2804443,82.0,28340000.0,['Drama'],R,['Frank Darabont'],"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...","['Over', 'the', 'course', 'of', 'several', 'ye..."
1,1,The Godfather,1972,175,9.2,1954174,100.0,134970000.0,"['Crime', ' Drama']",R,['Francis Ford Coppola'],"['Marlon Brando', 'Al Pacino', 'James Caan', '...","['Don', 'Vito', 'Corleone,', 'head', 'of', 'a'..."
2,2,Ramayana: The Legend of Prince Rama,1993,135,9.2,12995,NaN,NaN,"['Animation', ' Action', ' Adventure']",PG,"['Ram Mohan', 'Yûgô Sakô', 'Koichi Saski']","['Arun Govil', 'Nikhil Kapoor', 'Edie Mirman',...","['An', 'anime', 'adaptation', 'of', 'the', 'Hi..."
3,3,The Chaos Class,1975,87,9.2,42231,NaN,NaN,"['Comedy', ' Drama']",NaN,['Ertem Egilmez'],"['Kemal Sunal', 'Münir Özkul', 'Halit Akçatepe...","['Lazy,', 'uneducated', 'students', 'share', '..."
4,4,The Dark Knight,2008,152,9.0,2786129,84.0,534860000.0,"['Action', ' Crime', ' Drama']",PG-13,['Christopher Nolan'],"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...","['When', 'the', 'menace', 'known', 'as', 'the'..."


In [18]:
data.iterrows()

<generator object DataFrame.iterrows at 0x7b797ff0bb50>

In [19]:
for index, row in data.iterrows():
    # converte uma string que contem uma lista (como ['x', 'y']) de volta para uma lista de python real.
    # essa lista contem o nome dos atores
    actors = ast.literal_eval(row['Stars'])
    movie = row['Movie Name']  # nome do filme que conecta os dois atores

    # percorrem a lista de atores de forma que todos os pares possíveis de atores sejam considerados
    for i in range(len(actors)):
        for j in range(i + 1, len(actors)):
           # o nome da aresta é o nome do filme em comum
            G.add_edge(actors[i], actors[j], movie=movie)

# Busca Bidirecional

In [20]:
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt

# start é o ator de partida (kevin) e goal é o altor alvo
def bidirectional_search(graph, start, goal):
    if start == goal:
        return [start]

    # Inicialização
    front_start = deque([start])  # Fila da busca a partir do início
    front_goal = deque([goal])     # Fila da busca a partir do fim
    explored_start = {start}       # guarda os nós já visitados do início
    explored_goal = {goal}         # guarda os nós já visitados do fim
    parent_start = {start: None}   # Dicionário para reconstrução do caminho
    parent_goal = {goal: None}     # Dicionário para reconstrução do caminho

    while front_start and front_goal: #continua a busca enquanto tiver nós nas duas filas
        # Expansão da busca a partir do início
        current_start = front_start.popleft() # remove o primeiro no da fila p ser o nó atual
        for neighbor in graph.neighbors(current_start):
          # p cada vizinho do nó atual, verifica se ja foi visitado
            if neighbor not in explored_start:
                explored_start.add(neighbor) #adiciona aos visitados
                parent_start[neighbor] = current_start # guarda o nó pai p reconstruir o caminho dps
                front_start.append(neighbor) # guarda o vizinho na fila

                if neighbor in explored_goal:
                  # se o vizinho estiver nos explorados a partir do fim significa q há um caminho
                    return reconstruct_path(parent_start, parent_goal, neighbor)

        # Expansão da busca a partir do fim
        # funciona do msm jeito, so q começa pelo fim
        current_goal = front_goal.popleft()
        for neighbor in graph.neighbors(current_goal):
            if neighbor not in explored_goal:
                explored_goal.add(neighbor)
                parent_goal[neighbor] = current_goal
                front_goal.append(neighbor)

                # Verifica se encontrou um caminho
                if neighbor in explored_start:
                    return reconstruct_path(parent_start, parent_goal, neighbor)

    return None  # se as duas listas estiverem vazias e n tiver interseção, significa q n ha caminho entre os atores

#reconstrói o caminho completo de start ate goal pelo no de interseção (meeting_node)
def reconstruct_path(parent_start, parent_goal, meeting_node):
    path_from_start = []
    #começa no nó de intersec e vai subindo no dicion "parent_start" ate chegar ao nó de partida
    node = meeting_node
    while node is not None:
        path_from_start.append(node)
        node = parent_start[node]
    path_from_start.reverse()  # Inverter para obter a ordem correta

    # Reconstrução do caminho a partir do fim até o nó de interseção
    # faz a msm coisa mas pelo nó de intersec até o destino, sem inverter a ordem
    path_from_goal = []
    node = meeting_node
    while node is not None:
        path_from_goal.append(node)
        node = parent_goal[node]

    # retorna o caminho completo, unindo o caminho do inicio ate a intersec, e da intersec ate o goal
    return path_from_start + path_from_goal[1:]  # p não duplicar o nó de interseção


In [21]:
#exemplo
start_actor = "Kevin Bacon"  # Ator de partida
goal_actor = "Jackie Chan"   # Ator de destino

path = bidirectional_search(G, start_actor, goal_actor)

if path:
    print("Caminho encontrado:", " -> ".join(path))
else:
    print("Nenhum caminho encontrado.")


Caminho encontrado: Kevin Bacon -> Tim Robbins -> Claire Forlani -> Jackie Chan


# Busca A*

In [22]:
import heapq

def heuristic(actor1, actor2, graph):
    """Heurística simples: retorna a distância entre dois atores no grafo."""
    # p heuristica: retorna o número de conexões (grafo não ponderado)
    #atores mais conectados tem maior chance de estar no caminho mais curto
    return abs(len(list(graph.neighbors(actor1))) - len(list(graph.neighbors(actor2))))

# algoritmo A* p encontrar o caminho mais curto entre start e goal
def a_star_search(graph, start, goal):
    if start == goal:
        return [start]

    # fila de prioridade onde o prox ó a ser visitado eh oq tem menor custo total fn = custo + heurist
    frontier = []
    heapq.heappush(frontier, (0, start))  # começa com o nó inicial e custo 0

    # Dicionários para armazenar o custo de chegada e o caminho
    came_from = {start: None} #guarda de onde veio cada nó p recosntruir o caminho dps
    cost_so_far = {start: 0} #armazena o custo encontrado ate entao p cada nó

    while frontier:
        # Retira o nó com o menor custo estimado
        current_priority, current_node = heapq.heappop(frontier)

        # Se chegou no objetivo, reconstrói o caminho
        if current_node == goal:
            return reconstruct_path_a_star(came_from, current_node)

        # Explora os vizinhos do nó atual
        for neighbor in graph.neighbors(current_node):
            new_cost = cost_so_far[current_node] + 1  # Custo de cada aresta é 1 (pode ser modificado)
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost  # Atualiza o custo
                priority = new_cost + heuristic(neighbor, goal, graph)  # f(n) = g(n) + h(n)
                heapq.heappush(frontier, (priority, neighbor))  # Adiciona à fila com nova prioridade
                came_from[neighbor] = current_node  # Registra de onde veio o vizinho

    return None  # Se não encontrar um caminho

# dps de achar o nó objetivo, percorre-se o dic "came_from" p reconstruir o caminho o do goal ao start
# dps inverte  a ordem p obter o caminho correto
def reconstruct_path_a_star(came_from, current_node):
    path = []
    while current_node is not None:
        path.append(current_node)
        current_node = came_from[current_node]
    path.reverse()  # Inverte o caminho para a ordem correta
    return path

# Exemplo de uso
start_actor = "Kevin Bacon"
goal_actor = "Jackie Chan"

path = a_star_search(G, start_actor, goal_actor)

if path:
    print("Caminho encontrado (A*):", " -> ".join(path))
else:
    print("Nenhum caminho encontrado.")


Caminho encontrado (A*): Kevin Bacon -> Julia Roberts -> John Cusack -> Jackie Chan
